In [ ]:
import pandas as pd
import os
from os import listdir
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import trafilatura
import lxml_html_clean as cl
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from operator import itemgetter
from nltk.corpus import PlaintextCorpusReader
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.keys import Keys

stopwords=[]
current_directory = os.getcwd()
pathname = os.path.join(current_directory, "data/StopWords_Auditor.txt")


with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for word in re.findall(r'\w+', line):
            stopwords.append(word.lower())
            
                        
pathname = os.path.join(current_directory, "data/StopWords_Currencies.txt")

with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for word in re.findall(r'\w+', line):
            stopwords.append(word.lower())


pathname = os.path.join(current_directory, "data/StopWords_DatesandNumbers.txt")


with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for word in re.findall(r'\w+', line):
            stopwords.append(word.lower())

pathname = os.path.join(current_directory, "data/StopWords_Generic.txt")


with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for word in re.findall(r'\w+', line):
            stopwords.append(word.lower())
            
pathname = os.path.join(current_directory, "data/StopWords_GenericLong.txt")


with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for  word in re.findall(r'\w+', line):
            stopwords.append(word.lower())
            
pathname = os.path.join(current_directory, "data/StopWords_Geographic.txt")


with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for word in re.findall(r'\w+', line):
            stopwords.append(word.lower())

pathname = os.path.join(current_directory, "data/StopWords_Names.txt")


with open(pathname,'r') as file:
    #print(file)
    for line in file:
        #print(line)
        for word in re.findall(r'\w+', line):
            stopwords.append(word.lower())

negwords=[]

current_directory = os.getcwd()
pathname = os.path.join(current_directory, "data/negative-words.txt")


with open(pathname,'r') as file:
  
    for line in file:
      
        for word in re.findall(r'\w+', line):
            negwords.append(word.lower())
#print(negwords)

poswords=[]

current_directory = os.getcwd()
pathname = os.path.join(current_directory, "data/positive-words.txt")


with open(pathname,'r') as file:
   
    for line in file:
       
        for word in re.findall(r'\w+', line):
            poswords.append(word.lower())
#print(poswords)

#find if overlapping words

neg = set(negwords)
stop= set(stopwords)
pos = set(poswords)

#print(len(negwords)) #if overlapping in stop and negative remove from negative
intersected_set = stop.intersection(neg)
neg.discard(intersected_set)
negwords=list(neg)
#print(len(negwords))


#print(len(poswords))-- #if overlapping in negative and positive remove from positive
intersected_set = neg.intersection(pos)
pos.discard(intersected_set)
poswords=list(pos)
#print(len(poswords))

#print(len(poswords)) #if overlapping in stop and positive remove from positive
intersected_set = stop.intersection(pos)
pos.discard(intersected_set)
poswords=list(pos)
#print(len(poswords))


                        

current_directory = os.getcwd()

#open Input excel file , iterate through each row , extract the article and save to a text file saved with urlid

pathname = os.path.join(current_directory, "data\Input.xlsx")
excel=pd.read_excel(pathname)
urllst=[]

for key, row in excel.iterrows(): 
    url_id = row['URL_ID']
    url = row['URL']
    urllst.append([url_id,url])
    requests.get(url)
    response = requests.get(url)  
    if response.status_code == 200:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        wordlist=""
        element_list=""
        article=""
        downloaded = trafilatura.fetch_url(url)
        #time.sleep(5)
        element_list=trafilatura.extract(downloaded)
# loop through word elements to remove speacial characters like new line
        for element in element_list:
            wordlist+=element

        article=wordlist
        title=soup.find('title').text
        
        file_name = current_directory + '/data/articles/'+( url_id) + '.txt'
        with open(file_name, 'w',encoding='utf-8') as file:
            file.write(title + '\n' + article)
    else:
        file_name = current_directory + '/data/articles/'+( url_id) + '.txt'
        with open(file_name, 'w',encoding='utf-8') as file:
            file.write("")
            
urldf=pd.DataFrame(urllst,columns=['urlid','url'])     

def getscores(flname,path):
    
 #function that will open txt file score the article and prepare a list with accumulated score

    file_name = path + '/'+ flname
    filew = open(file_name, 'r',encoding='utf-8') 
    filep = open(file_name, 'r',encoding='utf-8') 
    print(file_name)
    search=word_tokenize(filew.read())
    para= sent_tokenize(filep.read())
    
    
    cleanword=0      # word count
    allword=0
    l=0              # line length
    s=0              # sentence count
    tl=0             # total line length
    prncnt=0
    wordlen=0
    positivescore=0
    negativescore=0
    totalsyllcnt=0
    lowersrch=[]
    nwsrch=[]
    compwords=[]
    polarityscore=0
    subjectivescore=0
    avgwrdperln=0
    fogindex=0
    syllcntperword=0
    avgwrdlen=0
    urlid=flname.split('.')[0]
    
    if len(search) == 0:
        print('No data in file ' + file_name)
        result.append([urlid, file_name,positivescore,negativescore,polarityscore,subjectivescore,fogindex,avgwrdperln,len(compwords),cleanword,syllcntperword,prncnt,avgwrdlen])
        return

    for x in para:
        l=len(x)
        s=s+1
        tl=tl+l

# Calculate complex word count, Clean word count, Pronoun word count- pronoun list expanded

    try:
        for word in search:
            word = re.sub('[^A-Za-z0-9]+', ' ', word)
            vowels = 'aeiou'
            syllcnt = sum( 1 for letter in word if letter.lower() in vowels)
            allword=allword+1
            wordlen=wordlen+len(word)
            totalsyllcnt=totalsyllcnt+syllcnt
            if syllcnt > 2:
                compwords.append(word)
            if word.casefold() not in stopwords:
                cleanword=cleanword+1
                nwsrch.append(word)
            lowersrch = list(map(lambda x : x.lower(), nwsrch))
            pronouns=['i','we','us','my','our','ours','they','them','him','her']
            if word.lower() in pronouns : prncnt=prncnt+1
    except:
           print('No Compound/Clean/Pronoun words ' + file_name) 


# Calculate positive score, negative score

    neg = set(negwords)
    stop= set(stopwords)
    pos = set(poswords)
    srch= set(lowersrch)

    
    try:
        intersected_set =pos.intersection(srch)
        intvalue=list(intersected_set )
        c = Counter(lowersrch)
        itemgetter(*intvalue)(c)
        positivescore=sum(itemgetter(*intvalue)(c))
    except:
        print('No Positive words ' + file_name)
        positivescore=0

    try:
        intersected_set =neg.intersection(srch)
        intvalue=list(intersected_set )
        c = Counter(lowersrch)
        #print(c)
        itemgetter(*intvalue)(c)
        #print(intersected_set,itemgetter(*intvalue)(c))
        negativescore=sum(itemgetter(*intvalue)(c))
    except:
        print('No negative words ' + file_name)
        negativescore=0

#Other Calculated fields
    
    wordcnt=allword
    percompwords=len(compwords)/allword*100
    #positivescore
    #negativescore
    polarityscore=(positivescore-negativescore)/(positivescore+negativescore+0.000001)
    subjectivescore=(positivescore+negativescore)/(allword+0.000001)
    avgwrdperln=allword/s
    fogindex=0.4*(avgwrdperln+percompwords)
    #compwords
    #cleanword
    syllcntperword=totalsyllcnt/allword
    #prncnt
    avgwrdlen=wordlen/allword
    
    
    result.append([urlid, file_name,positivescore,negativescore,polarityscore,subjectivescore,fogindex,avgwrdperln,len(compwords),cleanword,syllcntperword,prncnt,avgwrdlen])
    
    return 

result=[]
df = pd.DataFrame()
current_directory = os.getcwd()
path=current_directory+'/data/articles' 
for filename in listdir(path):
    getscores(filename,path)
df=pd.DataFrame(result,columns=['urlid','TxtFile Location','Positive Score','Negative Score','Polarity', 'Subjectivity','Analysis of Readability','Average No.of Words/Sentence','Complex Word Count','Word Count', 'Syllable Count/Word', 'Personal Pronouns', 'Avg Word Len'])

Output=pd.merge(df,urldf, how='inner',indicator=True, left_on='urlid',right_on='urlid')

savefilename=current_directory+'/data/Output.csv'
Output.to_csv(savefilename, sep=',')